In [11]:
# 필요한 라이브러리를 먼저 설치해야 합니다.
# 터미널 또는 명령 프롬프트에서: pip install requests python-dotenv
import requests
import io
import zipfile
import re
import os
import csv
from datetime import datetime
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
# 이 스크립트와 같은 폴더에 .env 파일이 없다면 주석 처리해도 됩니다.
# load_dotenv()

# DART API 키를 환경 변수에서 가져옵니다.
# .env 파일이 없다면 DART_API_KEY = "여기에_API_키_입력" 과 같이 직접 설정할 수 있습니다.
DART_API_KEY = os.environ.get('OPEN_DART_API')

# 분석할 회사 목록
companys = {
            '삼성전자': '005930', 'SK하이닉스': '000660', 'LG에너지솔루션': '373220', '삼성바이오로직스': '207940', 'KB금융': '105560', 
            '삼성전자우': '005935', '현대차': '005380', '기아': '000270', 'NAVER': '035420', '셀트리온': '068270', '두산에너빌리티': '034020', '한화에어로스페이스': '012450', 
            'HD현대중공업': '329180', '신한지주': '055550', '삼성물산': '028260', '현대모비스': '012330', '하나금융지주': '086790', '카카오': '035720', '삼성생명': '032830', 
            'HMM': '011200', 'POSCO홀딩스': '005490', '한국전력': '015760', '한화오션': '042660', 'SK스퀘어': '402340', 'HD한국조선해양': '009540', '삼성화재': '000810', 
            '메리츠금융지주': '138040', '현대로템': '064350', 'LG화학': '051910', '우리금융지주': '316140', '포스코인터내셔널': '047050',
            'SK이노베이션': '096770', '고려아연': '010130', '크래프톤': '259960', '기업은행': '024110', 'KT&G': '033780', 'HD현대일렉트릭': '267260', 'SK': '034730',
            '삼성중공업': '010140', '카카오뱅크': '323410', 'KT': '030200', '삼성SDI': '006400', '삼성에스디에스': '018260', 'LG': '003550', 'LIG넥스원': '079550', 
            '미래에셋증권': '006800', 'LG전자': '066570', 'SK텔레콤': '017670', '하이브': '352820', '삼양식품': '003230', '포스코퓨처엠': '003670', 'HD현대': '267250', 
            # '카카오페이': '377300', '현대글로비스': '086280', '삼성전기': '009150', '한화시스템': '272210', 'DB손해보험': '005830', '두산': '000150', '대한항공': '003490', 
            # '효성중공업': '298040', '한미반도체': '042700', '유한양행': '000100', '한국금융지주': '071050', 
            # 'HD현대마린솔루션': '443060', '한국항공우주': '047810', '한진칼': '180640', '현대건설': '000720', 'LS ELECTRIC': '010120', 'LG씨엔에스': '064400', '아모레퍼시픽': '090430', 
            # 'SK바이오팜': '326030', '코웨이': '021240', 'NH투자증권': '005940', 'HD현대미포': '010620', 'S-Oil': '010950', '한화': '000880', '삼성증권': '016360', '삼성카드': '029780', 
            # 'LG유플러스': '032640', '에이피알': '278470', '키움증권': '039490', '한화솔루션': '009830', 'LS': '006260', '현대차2우B': '005387', '맥쿼리인프라': '088980', 
            # '한국타이어앤테크놀로지': '161390', '두산밥캣': '241560', '넷마블': '251270', 'LG생활건강': '051900', 'JB금융지주': '175330', 'BNK금융지주': '138930', 
            # '삼성E&A': '028050', 'GS': '078930', '오리온': '271560', '현대제철': '004020', 'CJ': '001040', 'LG디스플레이': '034220', '현대오토에버': '307950', '엔씨소프트': '036570', 
            # '강원랜드': '035250', '한국가스공사': '036460', '이수페타시스': '007660', '두산로보틱스': '454910'
            }

# -----------------------------------------------------------------------------
# 1단계: 지정된 기간의 모든 사업보고서 목록 조회
# -----------------------------------------------------------------------------
def get_all_business_reports(api_key, corp_code, start_year, end_year):
    """지정한 기업의 특정 기간 내 모든 사업보고서(정기보고서) 및 관련 정정보고서 목록을 조회하여 반환합니다."""
    print(f"1단계: {corp_code} 기업의 {start_year}년~{end_year}년 사업보고서 관련 목록 조회를 시작합니다.")
    list_url = "https://opendart.fss.or.kr/api/list.json"
    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bgn_de': f'{start_year}0101',
        'end_de': f'{end_year}1231',
        'pblntf_ty': 'A',
        'pblntf_detail_ty': 'A001', # 사업보고서
        'page_no': '1',
        'page_count': '100'
    }
    try:
        res = requests.get(list_url, params=params)
        res.raise_for_status()
        data = res.json()
        if data.get('status') != '000':
            print(f"API 오류: {data.get('message')}")
            return []
        if not data.get('list'):
            print("조회된 사업보고서가 없습니다.")
            return []
        
        reports = [{'rcept_no': item.get('rcept_no'), 'report_nm': item.get('report_nm'), 'rcept_dt': item.get('rcept_dt')} for item in data.get('list', [])]
        # API 결과가 기본적으로 시간순이므로 reverse 하지 않아도 최신순으로 정렬됩니다.
        print(f"성공! 총 {len(reports)}개의 관련 보고서를 찾았습니다.")
        return reports
    except Exception as e:
        print(f"1단계 오류: {e}")
        return []

# -----------------------------------------------------------------------------
# 2단계: 접수번호를 이용해 공시서류 원본 파일(XML) 요청
# -----------------------------------------------------------------------------
def get_report_xml(api_key, rcept_no):
    """접수번호를 사용하여 공시서류 원본 파일(ZIP)을 요청하고, XML 내용을 반환합니다."""
    if not rcept_no: return None
    print(f"\n2단계: 접수번호 {rcept_no}의 공시서류 원본 파일 조회를 시작합니다.")
    document_url = "https://opendart.fss.or.kr/api/document.xml"
    params = {'crtfc_key': api_key, 'rcept_no': rcept_no}
    try:
        res = requests.get(document_url, params=params)
        res.raise_for_status()
        print("성공! 공시서류 원본 파일(ZIP)을 내려받았습니다.")
        
        with zipfile.ZipFile(io.BytesIO(res.content)) as zf:
            xml_file_name = next((f for f in zf.namelist() if f.endswith('.xml')), None)
            if not xml_file_name:
                print("ZIP 파일 내에서 XML 파일을 찾을 수 없습니다."); return None
            
            with zf.open(xml_file_name) as xml_file:
                content_bytes = xml_file.read()
                try:
                    return content_bytes.decode('utf-8-sig')
                except UnicodeDecodeError:
                    return content_bytes.decode('cp949', errors='ignore')
    except Exception as e:
        print(f"2단계 처리 중 예외가 발생했습니다: {e}"); return None

# -----------------------------------------------------------------------------
# 3단계: XML에서 모든 텍스트 추출 및 정리
# -----------------------------------------------------------------------------
def get_full_text_from_xml(xml_content):
    """XML/HTML 태그를 모두 제거하고 전체 텍스트를 정리하여 반환합니다."""
    print("3단계: 보고서 전체 내용에서 텍스트 추출을 시작합니다.")
    if not xml_content: return ""
    text = re.sub(r'<[^>]+>', ' ', xml_content)
    text = re.sub(r'&nbsp;', ' ', text); text = re.sub(r'&gt;', '>', text); text = re.sub(r'&lt;', '<', text); text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'(\s*\n\s*)+', '\n', text).strip()
    text = re.sub(r'\s{2,}', ' ', text)
    print("성공! 보고서 전체 텍스트를 정리했습니다.")
    return text

# -----------------------------------------------------------------------------
# 4단계: 보고서명 또는 접수일자에서 연도 추출
# -----------------------------------------------------------------------------
def extract_year(report_name, reception_date):
    """보고서명에서 연도를 우선 추출하고, 실패 시 접수일자에서 연도를 추출합니다."""
    # 정규표현식을 사용하여 보고서명에서 (YYYY.MM) 형식의 사업연도를 찾습니다.
    # 예: 사업보고서 (2023.12) -> 2023
    match = re.search(r'\((\d{4})', report_name)
    if match:
        return match.group(1)
    
    # 위에서 못찾으면, 접수일자(YYYYMMDD)에서 연도를 가져와 -1을 합니다. 
    # (보통 2024년 3월에 2023년도 사업보고서를 제출하기 때문)
    if reception_date:
        try:
            return str(int(reception_date[:4]) - 1)
        except (ValueError, TypeError):
            return "N/A" # 변환 실패 시
            
    return "N/A" # 연도를 찾을 수 없는 경우

# -----------------------------------------------------------------------------
# 5단계: 추출된 모든 보고서 내용을 하나의 CSV 파일로 저장
# -----------------------------------------------------------------------------
def save_combined_csv(reports_data, filename):
    """여러 보고서 데이터를 하나의 CSV 파일로 저장합니다."""
    print(f"\n5단계: 통합 CSV 저장을 시작합니다. 파일명: '{filename}'")
    try:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['년도', '회사명', '텍스트 미리보기'])
            for report in reports_data:
                writer.writerow([report['year'], report['name'], report['content']])
        
        print(f"성공! 통합 CSV '{filename}' 파일 저장을 완료했습니다.")
    except Exception as e:
        print(f"오류: 통합 CSV 파일 저장 중 문제가 발생했습니다. ({filename})")
        print(f"상세 오류: {e}")

# -----------------------------------------------------------------------------
# 메인 실행 부분
# -----------------------------------------------------------------------------
start_year,end_year=2022,2024

if __name__ == "__main__":
    if not DART_API_KEY:
        print("오류: DART API 키가 설정되지 않았습니다. .env 파일을 확인하거나 코드에 직접 입력해주세요.")
    else:
        output_folder = "사업보고서_CSV_통합"
        os.makedirs(output_folder, exist_ok=True)
        print(f"결과가 ./{output_folder}/ 폴더에 저장됩니다.")
        
        start_year,end_year=2022,2024
        print(f"조회 기간: {start_year}년 1월 1일 ~ {end_year}년 12월 31일")

        all_companies_reports = []

        for name, code in companys.items():
            print("\n" + "="*80)
            print(f"[{name}({code})] 기업의 보고서 처리를 시작합니다.")
            print("="*80)
            
            all_reports_meta = get_all_business_reports(DART_API_KEY, code, start_year, end_year)

            if not all_reports_meta:
                print(f"[{name}] 처리할 보고서가 없어 다음 기업으로 넘어갑니다.")
                continue

            # --- [핵심 수정] 연도별로 최종 보고서만 선택하는 로직 ---
            final_reports_to_process = {}
            for report_meta in all_reports_meta:
                year = extract_year(report_meta['report_nm'], report_meta['rcept_dt'])
                if year != "N/A":
                    # 딕셔너리에 연도를 key로 하여 보고서 정보를 저장합니다.
                    # 같은 연도의 보고서가 여러 개 있으면, 나중에 나온 것(최신)으로 덮어쓰게 됩니다.
                    final_reports_to_process[year] = report_meta
            
            print(f"\n총 {len(final_reports_to_process)}개의 최종 사업보고서를 처리합니다: {sorted(final_reports_to_process.keys())}년도")
            # ---------------------------------------------------------

            for year, report_meta in sorted(final_reports_to_process.items()):
                print("\n" + "-"*70)
                print(f"처리 대상: {year}년도 사업보고서 ({report_meta['report_nm']}, 접수번호: {report_meta['rcept_no']})")
                
                report_xml_content = get_report_xml(DART_API_KEY, report_meta['rcept_no'])
                
                if report_xml_content:
                    full_report_text = get_full_text_from_xml(report_xml_content)
                    
                    if full_report_text:
                        all_companies_reports.append({
                            'year': year,
                            'name': name,
                            'content': full_report_text
                        })
                    else:
                        print("보고서 내용이 비어있어 처리를 건너뜁니다.")
        
        if all_companies_reports:
            final_csv_filename = os.path.join(output_folder, f"통합_사업보고서 ({start_year}-{end_year}).csv")
            save_combined_csv(all_companies_reports, final_csv_filename)
        else:
            print("\n저장할 보고서 내용이 없어 통합 CSV 파일을 생성하지 않았습니다.")

        print("\n\n모든 기업의 보고서 처리를 완료했습니다.")


결과가 ./사업보고서_CSV_통합/ 폴더에 저장됩니다.
조회 기간: 2022년 1월 1일 ~ 2024년 12월 31일

[삼성전자(005930)] 기업의 보고서 처리를 시작합니다.
1단계: 005930 기업의 2022년~2024년 사업보고서 관련 목록 조회를 시작합니다.
성공! 총 12개의 관련 보고서를 찾았습니다.

총 4개의 최종 사업보고서를 처리합니다: ['2021', '2022', '2023', '2024']년도

----------------------------------------------------------------------
처리 대상: 2021년도 사업보고서 (사업보고서 (2021.12), 접수번호: 20220308000798)

2단계: 접수번호 20220308000798의 공시서류 원본 파일 조회를 시작합니다.
성공! 공시서류 원본 파일(ZIP)을 내려받았습니다.
3단계: 보고서 전체 내용에서 텍스트 추출을 시작합니다.
성공! 보고서 전체 텍스트를 정리했습니다.

----------------------------------------------------------------------
처리 대상: 2022년도 사업보고서 (분기보고서 (2022.03), 접수번호: 20220516001751)

2단계: 접수번호 20220516001751의 공시서류 원본 파일 조회를 시작합니다.
성공! 공시서류 원본 파일(ZIP)을 내려받았습니다.
3단계: 보고서 전체 내용에서 텍스트 추출을 시작합니다.
성공! 보고서 전체 텍스트를 정리했습니다.

----------------------------------------------------------------------
처리 대상: 2023년도 사업보고서 (분기보고서 (2023.03), 접수번호: 20230515002335)

2단계: 접수번호 20230515002335의 공시서류 원본 파일 조회를 시작합니다.
성공! 공시서류 원본 파일(ZIP)을 내려받았습니다.
3단계: 보고서 전체 내용에서 텍

In [32]:
import pandas as pd
import os

folder = "사업보고서_CSV_통합"
file = "통합_사업보고서 (2022-2024).csv"
file_path = os.path.join(folder, file)

target_company = '삼양식품'
year = 2022

df = pd.read_csv(file_path)

df['년도'] = pd.to_numeric(df['년도'])
            
target_report = df[(df['회사명'] == target_company) & (df['년도'] == year)]

report_content = target_report['텍스트 미리보기'].iloc[0]
    
print(report_content[100000:110000])



권 미상환 잔액
(기준일 :
2022년 03월 31일
)
(단위 : 원)
잔여만기
10일 이하
10일초과&cr;30일이하
30일초과&cr;90일이하
90일초과&cr;180일이하
180일초과&cr;1년이하
1년초과&cr;2년이하
2년초과&cr;3년이하
3년 초과
합 계
미상환 잔액
공모
-
-
-
-
-
-
-
-
-
사모
-
-
-
-
-
-
-
-
-
합계
-
-
-
-
-
-
-
-
-
단기사채 미상환 잔액
(기준일 :
2022년 03월 31일
)
(단위 : 원)
잔여만기
10일 이하
10일초과&cr;30일이하
30일초과&cr;90일이하
90일초과&cr;180일이하
180일초과&cr;1년이하
합 계
발행 한도
잔여 한도
미상환 잔액
공모
-
-
-
-
-
-
-
-
사모
-
-
-
-
-
-
-
-
합계
-
-
-
-
-
-
-
-
회사채 미상환 잔액
(기준일 :
2022년 03월 31일
)
(단위 : 원)
잔여만기
1년 이하
1년초과&cr;2년이하
2년초과&cr;3년이하
3년초과&cr;4년이하
4년초과&cr;5년이하
5년초과&cr;10년이하
10년초과
합 계
미상환 잔액
공모
-
-
75,000,000,000
-
-
-
-
75,000,000,000
사모
-
-
-
-
-
-
-
-
합계
-
-
75,000,000,000
-
-
-
-
75,000,000,000
신종자본증권 미상환 잔액
(기준일 :
2022년 03월 31일
)
(단위 : 원)
잔여만기
1년 이하
1년초과&cr;5년이하
5년초과&cr;10년이하
10년초과&cr;15년이하
15년초과&cr;20년이하
20년초과&cr;30년이하
30년초과
합 계
미상환 잔액
공모
-
-
-
-
-
-
-
-
사모
-
-
-
-
-
-
-
-
합계
-
-
-
-
-
-
-
-
조건부자본증권 미상환 잔액
(기준일 :
2022년 03월 31일
)
(단위 : 원)
잔여만기
1년 이하
1년초과&cr;2년이하
2년초과&cr;3년이하
3년초과&cr;4년이하
4년초과&cr;5년이하
5년초과&cr;10년이하
10년초과&cr;2

In [29]:
a = pd.read_csv('C:\파이썬\data\사업보고서_CSV_통합\통합_사업보고서 (2022-2024).csv')
hyundai = a[a['회사명']=='SK하이닉스']
hyundai

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\iamkk\AppData\Local\Temp\ipykernel_30200\764828560.py:1: SyntaxWarning: invalid escape sequence '\d'
  a = pd.read_csv('C:\파이썬\data\사업보고서_CSV_통합\통합_사업보고서 (2022-2024).csv')


,년도,회사명,텍스트 미리보기
4,2021,SK하이닉스,연결감사보고서\n3.4\n에스케이하이닉스주식회사\n00260295\n10000000...
5,2022,SK하이닉스,분기보고서\n4.8\nSK하이닉스\n분 기 보 고 서\n&cr;&cr;&cr;\n(...
6,2023,SK하이닉스,분기보고서\n5.0\nSK하이닉스\n분 기 보 고 서\n(제 76 기)\n사업연도\...
7,2024,SK하이닉스,분기보고서\n5.6\nSK하이닉스\n1\nY\n분 기 보 고 서\n(제 77 기)\...
